In [1]:
import polars as pl

In [2]:
df1 = pl.read_csv("./generated_data_01.csv")
df2 = pl.read_csv("./generated_data_02.csv")

In [3]:
df_cross_join = df1.join(df2, how="cross")

In [4]:
df_cross_join.head()

SKU,description,price,SKU_right,description_right,price_right
str,str,f64,str,str,f64
"""335-86-8164""","""Multi-channeled coherent proje…",851.89,"""223-82-2234""","""Fully-configurable value-added…",664.82
"""335-86-8164""","""Multi-channeled coherent proje…",851.89,"""750-22-2868""","""Right-sized 24/7 encryption""",540.28
"""335-86-8164""","""Multi-channeled coherent proje…",851.89,"""273-70-4025""","""Visionary context-sensitive su…",847.64
"""335-86-8164""","""Multi-channeled coherent proje…",851.89,"""355-71-1875""","""Ameliorated optimizing hierarc…",217.98
"""335-86-8164""","""Multi-channeled coherent proje…",851.89,"""504-41-5222""","""Synchronized composite capabil…",471.71


In [8]:
# df_cross_join.head().map_rows(lambda row: print(row))
from thefuzz import fuzz
def fuzzy_similarity(x, y):
    return fuzz.ratio(x, y)
#
df_cross_join.with_columns(
    similarity_ratio=pl.struct("description", "description_right")
            .map_batches(
                lambda combined: fuzzy_similarity(combined.struct.field("description"), combined.struct.field("description_right")),
                is_elementwise=True
            )
    )

In [4]:
# Add two arrays together:
# @guvectorize([(int64[:], int64[:], float64[:])], "(n),(n)->(n)")
def add(arr, arr2):
    #result = [0]*len(arr)
    print(f"{arr}, {arr2}")
    return arr + arr2

df3 = pl.DataFrame({"values1": [1, 2, 3], "values2": [10, 20, 30]})

df3.with_columns(
    # Create a struct that has two columns in it:
    plugs=pl.struct("values1", "values2")
            .map_batches(
                lambda combined: add(combined.struct.field("values1"), combined.struct.field("values2"))
            )
    )

shape: (3,)
Series: 'values1' [i64]
[
	1
	2
	3
], shape: (3,)
Series: 'values2' [i64]
[
	10
	20
	30
]


values1,values2,plugs
i64,i64,i64
1,10,11
2,20,22
3,30,33
